# Initialize a variable

## constant initializer

constant是常量张量，在生成时是必须有初始值的，不同于一般的tensor。tf.constant()与tf.constant_initializer()作用似乎是完全相同的，但实际上不同，tf.constant()是一个常量张量，可以用tf.Session()来执行；tf.constant_initializer()是一个张量，虽然生成时也必须给初始值，但作用是用来给variable提供初始化，是不能用tf.Session()来执行的。

    d = tf.Variable(1, name="v", expected_shape=[1])

    init = np.random.rand(1, 2)
    tf.get_variable('var_name', initializer=tf.constant(init))

    init = np.random.rand(1, 2)
    tf.get_variable('var_name', initializer=tf.constant_initializer(init))
    
weight is a float32, a contant in python   
    
    diag = tf.Variable([weight], trainable=True)
    diag_before = [0.0]*idx
    diag_before = tf.constant(diag_before)
    diag_after = [0.0]*(d-1-idx)
    diag_after = tf.constant(diag_after)
    row = tf.concat(values=[diag_before, diag, diag_after], axis=0)

在具体使用中，如果用的是tf.constant_initializer()，在使用tf.get_variable()的时候一般同时给出shape属性，如果用的是tf.constant()，在使用tf.get_variable()的时候就不能给shape属性

    word_embeddings = tf.get_variable(
        'word_embeddings',
        shape=(vocab.size(), vocab.embed_dim),
        initializer=tf.constant_initializer(vocab.embeddings),
        trainable=True
    )
    
## random uniform initializer

tf.random_uniform(): Generate A Random Tensor In Ten
    
## random normal initializer

tf.random_normal is a Tensor; buttf.random_normal_initializer is a RandomNormal initializer, not a Tensor. I think simple code best clarifies the difference between these two:

    # Simple examples to clarify tf.random_normal from tf.random_normal_initializer
    tf.reset_default_graph()
    # OP's code
    t4 = tf.get_variable('t4', initializer=tf.random_normal((2,), seed=0))
    t5 = tf.get_variable('t5', shape=(2,), initializer=tf.random_normal_initializer(seed=0))
    # clarifying Tensor vs Initializer outside the context of get_variable.
    t6 = tf.random_normal((2,),seed=0)
    t7 = tf.random_normal_initializer(seed=0)
    # types
    print(type(t6)) # <class 'tensorflow.python.framework.ops.Tensor'>
    print(type(t7)) # <class 'tensorflow.python.ops.init_ops.RandomNormal'>
    # run the graph...
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # OP's code
        print(sess.run(t4)) #[-0.39915761  2.10443926]
        print(sess.run(t5)) #[-0.39915761  2.10443926]
        # tf.random_normal is a Tensor
        print(sess.run(t6)) #[-0.39915761  2.10443926]
        # tf.random_normal_initializer returns a tf.RandomNormal, not a Tensor or Op, so can't be sess.run()!
        try:
            print(sess.run(t7)) # Exception!
        except:
            print("Exception!")
    # But notice that you don't need to initialize an initializer, just a variable.
    t8 = tf.random_normal_initializer(seed=0)
    t9 = tf.get_variable('t9',shape=(2,), initializer=t8)
    sess.run(t9.initializer)  # still need to initialize the variable
    print(sess.run(t9)) #[-0.39915761  2.10443926]
       
       
## truncated normal initializer

避免了长尾的初始值，使初始值集中在0的周围，矩阵乘法以后，在进sigmoid function之前，可以保证大多数值在0的周围，这样，再做back propagation时梯度较大，不会出现梯度消失

    weight = tf.Variable(tf.truncated_normal([conv_ksize[0], conv_ksize[1], x_tensor.shape[3].value, conv_num_outputs], mean=0, stddev=0.1))
    bias = tf.Variable(tf.truncated_normal([conv_num_outputs], mean=0, stddev=0.1))
 
    
## xavier_initializer

用来初始化全连接层的系数和bias  
Xavier and Yoshua's method, paper:   
Understanding the difficulty of training deep feedforward neural networks  
http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf

    initializer = tf.contrib.layers.xavier_initializer()
    w1 = tf.Variable(initializer(w1_shape))
    b1 = tf.Variable(initializer(b1_shape))
    
    
## tf.global_variables_initializer() and tf.tables_initializer()

tf.tables_initializer()的使用请见[similarity_fit_v3](https://github.com/mediaProduct2017/tf-gpu/blob/master/similarity_fit_v3.py)

在使用sess.run(tf.global_variables_initializer())前，要保证所需要的variable已经给出了图的定义，否则没法正常的变量初始化。


在高级接口中，不需要专门做variable的initialize，甚至variable都不需要自己定义，但实际上，是有varaible initialize这个过程的，一般是有的是类似random normal的方法，初始值集中分布在0的周围，都是很小的值。
